In [1]:
from cooking_bot.data_formats import *
from cooking_bot.encoders import get_clip_text_embedding, get_image_embedding, get_sentence_embedding, CLIP_MODEL, SENTENCE_TRANSORMER
from cooking_bot import REPO_PATH
from glob import glob
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import torch
import bertviz
from IPython.display import display
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModel
import os
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
os.chdir(REPO_PATH)

/home/felix/Documents/web/project


In [2]:
recipes = []

for id, path in enumerate(tqdm(glob("jsons/*.json"), "Filling Index")):
    
    with open(path, "r") as f:

        data = f.read()
    
    recipe = Recipe.model_validate_json(data)
    recipes.append(recipe)

Filling Index: 100%|██████████| 994/994 [00:32<00:00, 30.66it/s] 


In [3]:
many_instructions = [i for i in recipes if len(i.instructions) > 6]

def number_of_images(x : Recipe):
    
    s = 0
    
    for i in x.instructions:
        for img in i.stepImages:
            s += img.embedding is not None
            
    return s




many_instructions.sort(key=number_of_images, reverse=True)

len(many_instructions)

291

In [4]:
recipe : Recipe = many_instructions[6]



number_of_images(recipe), len(recipe.instructions)

(11, 12)

In [5]:
del recipes
    
del many_instructions

In [32]:
sentence_1 = "I do not have olive oil"
sentence_2 = "You can also use butter"

In [25]:
model_path = "sentence-transformers/all-MiniLM-L6-v2"
sentence_tokenizer = AutoTokenizer.from_pretrained(model_path)
sentence_config = AutoConfig.from_pretrained(model_path,  output_hidden_states=True, output_attentions=True, )  
sentence_model = AutoModel.from_pretrained(model_path, config=sentence_config)

In [37]:
sentence = sentence_1 +". "+ sentence_2
print(sentence)
sentence_tokens = sentence_tokenizer(sentence, return_tensors='pt',max_length = 512, padding=True, truncation = True )
with torch.no_grad():
    sentence_outputs = sentence_model(**sentence_tokens)

len(sentence_outputs["attentions"])

I do not have olive oil. You can also use butter


6

In [38]:
tokens = sentence_tokenizer.convert_ids_to_tokens(sentence_tokens["input_ids"][0]) 
attentions = sentence_outputs["attentions"]


bertviz.head_view(attentions, tokens, layer = 0, heads =(1,))


<IPython.core.display.Javascript object>

In [21]:
cross_encoder_path = 'cross-encoder/nli-distilroberta-base'

cross_tokenizer = AutoTokenizer.from_pretrained(cross_encoder_path)
cross_config = AutoConfig.from_pretrained(cross_encoder_path, output_hidden_states=True, output_attentions=True,output_cros_attentions = True)
cross_model = AutoModel.from_pretrained(cross_encoder_path, config=cross_config)


sentence_pair_tokens = cross_tokenizer(sentence_1, sentence_2, return_tensors='pt', truncation=True, padding=True)

# Generate model outputs
with torch.no_grad():
    cross_outputs = cross_model(**sentence_pair_tokens)
cross_outputs.keys()

Some weights of RobertaModel were not initialized from the model checkpoint at cross-encoder/nli-distilroberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states', 'attentions'])

In [23]:
# Accessing tokens and attentions
tokens = cross_tokenizer.convert_ids_to_tokens(sentence_pair_tokens["input_ids"][0])
attentions = cross_outputs["attentions"]

# Visualize attentions for the first layer, first head
bertviz.head_view(attentions, tokens, layer=0, heads=(1,))

Number of attention layers: 6


<IPython.core.display.Javascript object>